# 🪣 Cloud Data Access

<br />

## NASA's migration from "on-premise" to cloud

```{image} ./images/DAAC_map_new.jpg
:width: 700px
:align: center
```
<p style="font-size: 10px">image src: https://asf.alaska.edu/about-asf-daac/</p>

NASA has 12 Distributed Active Archive Centers (DAACs). Each DAAC is associated with a few sub-disciplines of Earth science, and those specialties correspond to which missions and data products those DAACs are in charge of. For example, LPDAAC is the land processes DAAC and is in charge of the Harmonized Landsat Sentinel (HLS) Product which is often used for land classification. Up until about 6 years ago (which is about when I started working with NASA), all NASA Earth Observation archives resided "on-premise" at the data center's physical locations in data centers they manage.

NASA, anticipating the exponential growth in their Earth Observation data archives, started the [Earthdata Cloud](https://www.earthdata.nasa.gov/eosdis/cloud-evolution) initiative. Now, NASA DAACs are in the process of migrating their collections to cloud storage. Existing missions are growing their collections as well, but new missions such as NISAR and SWOT are or will be the most significant contributors to NASA's archival volume growth.


```{image} ./images/archive-growth-FY22.jpg
:width: 900px
:align: center
```
<p style="font-size: 10px">image src: https://www.earthdata.nasa.gov/esds/esds-highlights/2022-esds-highlights</p>

Now, high priority and new datasets are being stored on **cloud object storage**.

<br />

## What is cloud object storage?

Cloud object storage stores unstructured data in a flat structure, called a bucket in AWS, where each object is identified with a unique key. The simple design of cloud object storage enables near infinite scalability. Object storage is distinguished from a database which requires database management system software to store data and often has connection limits. Object storage is distinct from file storage because files are stored in a hierarchical format and a network is not always required. Read more about cloud object storage and how it is different from other types of storage [in the AWS docs](https://aws.amazon.com/what-is/object-storage/).

```{image} ./images/s3-bucket-with-objects.png
:width: 150px
:align: left
```

Cloud object storage is accessible over the internet. If the data is public, you can use an HTTP link, but more typically you will use the cloud object storage protocol, such as `s3://path/to/file.text` along with some credentials to access the data. Access over the network is critical because it means many servers can access data in parallel and these storage systems are designed to be infinitely scalable and always available.

Popular libraries to access data on S3 are [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) and [s3fs](https://s3fs.readthedocs.io/).

```{image} ./images/cloud-and-local.png
:width: 500px
:align: center
```

:::{dropdown} 🏋️‍♀️ Exercise: Which ICESat-2 Datasets are on Earthdata Cloud?
:open:

Navigate [https://search.earthdata.nasa.gov](https://search.earthdata.nasa.gov), search for ICESat-2 and answer the following questions:

* Which DAAC hosts ICESat-2 datasets?
* How many ICESat-2 datasets are hosted on the AWS Cloud and how can you tell?
:::


## There are different access patterns, it can be confusing! 🤯

Here are a likely few:
1. Download data from a DAAC to your local machine.
2. Download data from cloud storage to your local machine.
3. Login to a virtual machine in the cloud, like CryoCloud, and download data from a DAAC.
4. Login to a virtual machine in the cloud and access data directly using a cloud object protocol, like s3.

```{image} ./images/different-modes-of-access.png
:width: 1000px
:align: center
```

:::{dropdown} 🤔 Which should you chose and why?
  You should use option 4 - direct access. Because S3 is a cloud service, egress (files being download outside of AWS services) is not free.
  **For data on Earthdata Cloud, you can only directly access files on S3 if you are in the same AWS region as the data. This is so NASA can avoid egress fees 💸 but it also benefits you because this style of access is much faster.**
  The good news is that CryoCloud is located in AWS us-west-2, the same region as NASA's Earthdata Cloud datasets!

  Of course, you may still need to access datasets from on-prem servers as well.

  <h3>Caveats</h3>
  <ul>
  <li>Direct S3 access could refer to either copying a whole file using the S3 protocol OR using lazy loading and reading just a portion of the file and the latter usually only performs well for cloud-optimized files.</li>
  <li>Having local file system access will always be faster than reading all or part of a file over a network, even in region (although S3 access is getting blazingly fast!) But you have to download the data, which is slow. You can also download objects from object storage onto a file system mounted onto a virtual machine, which would result in the fastest access and computation. But before architecting your applications this way, consider the tradeoffs of reproducibility (e.g. you'll have to move the data ever time), cost (e.g. storage volumes can be more expensive than object storage) and scale (e.g. there is usually a volume size limit, except in the case of [AWS Elastic File System](https://aws.amazon.com/efs/) which is even more pricey!).</li>

  When might you use option 3?
      
  </ul>
:::

:::{dropdown} 🏋️‍♀️ Bonus Exercise: Comparing time to copy data from S3 to CryoCloud with time to download over HTTP to your personal machine

Note: You will need URS credentials handy to do this exercise. You will need to store them in a local ~/.netrc file as instructed [here](https://urs.earthdata.nasa.gov/documentation/for_users/data_access/curl_and_wget)

```python
earthaccess.login()

aws_creds = earthaccess.get_s3_credentials(daac='NSIDC')

s3 = s3fs.S3FileSystem(
    key=aws_creds['accessKeyId'],
    secret=aws_creds['secretAccessKey'],
    token=aws_creds['sessionToken'],
)

results = earthaccess.search_data(
    short_name="ATL03",
    cloud_hosted=True,
    count=1
)

direct_link = results[0].data_links(access="direct")[0]
direct_link
```

Now time the download:
```python
%%time
s3.download(direct_link, lpath=direct_link.split('/')[-1])
```

Compare this with the time to download from HTTPS to your personal machine.

First, get the equivalent HTTPS URL:

```python
http_link = results[0].data_links()[0]
http_link
```

Then, copy and paste the following into a shell prompt, replacing `http_link` with the string from the last command. You will need to follow the instructions [here](https://urs.earthdata.nasa.gov/documentation/for_users/data_access/curl_and_wget) for this to work!

```bash
$ time curl -O -b ~/.urs_cookies -c ~/.urs_cookies -L -n {http_link}
```

For me, the first option, direct, in-region access took 11.1 seconds and HTTPS to personal machine took 1 minute and 48 seconds. The second value will depend on your wifi network.
:::

## Cloud vs Local Storage

:::{list-table}
:header-rows: 1

*   - Feature
    - Local
    - Cloud
*   - Scalability
    - ❌ limited by physical hardware
    - ✅ highly scalable
*   - Accessibility
    - ❌ access is limited to local network or complex setup for remote access
    - ✅ accessible from anywhere with an internet connection
*   - Collaboration
    - ❌ sharing is hard
    - ✅ sharing is possible with tools for access control
*   - Data backup
    - ❌ risk of data loss due to hardware failure or human error
    - ✅ typically includes redundancy ([read more](https://docs.aws.amazon.com/AmazonS3/latest/userguide/DataDurability.html))
*   - Performance
    - ✅ faster since it does not depend on any network
    - ❌ performance depends on internet speed or proximity to the data
:::


:::{admonition} Takeaways

1. NASA datasets are still managed by DAACs, even though many datasets are moving to the cloud.
2. Users are encouraged to access the data directly in the cloud through AWS services (like cryocloud!)
:::